# GMM error has some dimension error

**Fix it!**

In [1]:
import numpy as np
import dill
import pandas as pd
from scipy import optimize as opt
import time
import sys
sys.path.append('../')
import src

#GMM parameters
maxiters = 50 #120. About 2 minutes per iteration
time_periods = 40 #Maximum spell_t to consider
min_periods = 3 #What

#Check if this parameters still make sense for the current product
β10, β11 = -2, 3.5
β20, β21 = 1.3, -2.
betas = [β10, β11, β20, β21]


#Load policy and value function
#####################
file_n = "2018-10-5vfi_dict.dill" #Personal Macbook
#file_n = "2019-2-16vfi_dict.dill" #Work Macbook
with open('../data/' + file_n, 'rb') as file:
    data_d = dill.load(file)

lambdas = src.generate_simplex_3dims(n_per_dim=data_d['n_of_lambdas_per_dim'])
price_grid = np.linspace(data_d['min_price'], data_d['max_price'])

policy = data_d['policy']
valueF = data_d['valueF']
lambdas_ext = src.generate_simplex_3dims(n_per_dim=
                                         data_d['n_of_lambdas_per_dim'])

#Interpolate policy (level price). valueF is already a function
policyF = src.interpolate_wguess(lambdas_ext, policy)


#dataframe and standard deviation
cleaned_data = "../../firm_learning/data/cleaned_data/"

df = pd.read_csv(cleaned_data + "medium_prod_for_gmm.csv")
std_devs = (df.groupby('firm').level_prices.rolling(window=4, min=3)
            .std().reset_index()
            .rename(columns={'level_1': 't', 'level_prices': 'std_dev_prices'}))

df = pd.merge(df, std_devs, on=['firm', 't'], how='left')

mean_std_observed_prices = df.groupby('t').std_dev_prices.mean()[min_periods:]

xs = df.groupby('firm').xs.first().values
Nfirms = len(xs)
# Just add a zeroes. Makes sense for the gmm estimation
prior_shocks = src.gen_prior_shocks(Nfirms, σerror=0)




In [55]:
from src import from_theta_to_lambda_for_all_firms

θ  = [0.1, 2.1, -1, -2.1] 
xs_stand = np.abs(0.2*(xs - np.mean(xs))/ (np.std(xs)))
print(np.mean(xs_stand), np.std(xs_stand))
lambdas0 = from_theta_to_lambda_for_all_firms(θ, xs_stand, prior_shocks)
lambdas0[12:18]

0.12341207262308296 0.15738316406425085


array([[0.01447739, 0.2219079 , 0.76361472],
       [0.01495071, 0.22468458, 0.76036471],
       [0.01746401, 0.22573515, 0.75680084],
       [0.00804955, 0.19027017, 0.80168028],
       [0.00550311, 0.21259868, 0.7818982 ],
       [0.0165288 , 0.23415864, 0.74931256]])

In [81]:
mean_std_observed_prices_cl.index

Float64Index([ 3.0,  4.0,  5.0,  6.0,  7.0,  8.0,  9.0, 10.0, 11.0, 12.0, 13.0,
              14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0,
              25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0,
              36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0,
              47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0,
              58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0,
              69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0,
              80.0],
             dtype='float64', name='t')

In [86]:
# Fit t to observed_prices
#mean_std_expected_prices 
mean_std_observed_prices_cl = mean_std_observed_prices[pd.notnull(mean_std_observed_prices)]
mean_std_expected_prices_cl = mean_std_expected_prices[pd.notnull(mean_std_expected_prices)]

index_inters = np.intersect1d(mean_std_observed_prices_cl.index,
                              mean_std_expected_prices_cl.index)

mean_std_observed_prices_cl = mean_std_observed_prices_cl.loc[index_inters]

In [89]:
mean_std_observed_prices_cl.head(10)

t
3.0     0.000000e+00
4.0     0.000000e+00
5.0     0.000000e+00
6.0     0.000000e+00
7.0     0.000000e+00
8.0     5.050000e-01
9.0     5.831238e-01
10.0    5.050000e-01
11.0    4.904573e-08
12.0    4.904573e-08
Name: std_dev_prices, dtype: float64

In [90]:
mean_std_observed_prices.head(10)

t
3.0     0.000000e+00
4.0     0.000000e+00
5.0     0.000000e+00
6.0     0.000000e+00
7.0     0.000000e+00
8.0     5.050000e-01
9.0     5.831238e-01
10.0    5.050000e-01
11.0    4.904573e-08
12.0    4.904573e-08
Name: std_dev_prices, dtype: float64

In [79]:

w = None
t = len(mean_std_expected_prices)
if w is None:
    w = np.identity(t)
g = (1 / t) * (mean_std_expected_prices - mean_std_observed_prices[0:76])[:, np.newaxis]
(g.T @ w @ g)[0, 0]

nan

In [ ]:

mean_std_expected_prices = generate_mean_std_pricing_decisions(df, policyF,
                                                               lambdas0, min_periods)
    try:
        assert len(mean_std_observed_prices) == len(mean_std_expected_prices)
    except AssertionError as e:
        e.args += (len(mean_std_observed_prices), len(mean_std_expected_prices))
        raise
    t = len(mean_std_expected_prices)
    if w is None:
        w = np.identity(t)
    g = (1 / t) * (mean_std_expected_prices - mean_std_observed_prices)[:, np.newaxis]
    return (g.T @ w @ g)[0, 0]

src.gmm_error = gmm_error

In [58]:
def generate_mean_std_pricing_decisions(df, policyF, lambdas_at_0, min_periods=3):
    """
    Lambdas0: starting priors for each of the N firms
    """
    pricing_decision_dfs = []
    for i, firm in enumerate(df.firm.unique()):
        prices = src.generate_pricing_decisions(policyF, lambdas_at_0[i],
                                                df[df.firm == firm].log_dmd.values)
        pricing_decision_dfs.append(pd.DataFrame({'level_prices': prices,
                                                  'firm': np.repeat(firm, len(prices))
                                                  }))

    pricing_decision_df = pd.concat(pricing_decision_dfs, axis=0)

    std_dev_df = (pricing_decision_df.groupby('firm').level_prices.rolling(window=4,
                                                                           min=min_periods)
                  .std().reset_index()
                  .rename(columns={'level_1': 't',
                                   'level_prices': 'std_dev_prices'}))
    return std_dev_df.groupby('t').std_dev_prices.mean()[min_periods:]

mean_std_expected_prices = generate_mean_std_pricing_decisions(df, policyF,
                                                                   lambdas0, min_periods)

In [68]:
len(mean_std_observed_prices), len(mean_std_expected_prices)

(78, 76)

In [66]:
mean_std_observed_prices.head(10)

t
3.0     0.000000e+00
4.0     0.000000e+00
5.0     0.000000e+00
6.0     0.000000e+00
7.0     0.000000e+00
8.0     5.050000e-01
9.0     5.831238e-01
10.0    5.050000e-01
11.0    4.904573e-08
12.0    4.904573e-08
Name: std_dev_prices, dtype: float64

In [76]:
mean_std_observed_prices.tail(10)

t
71.0   NaN
72.0   NaN
73.0   NaN
74.0   NaN
75.0   NaN
76.0   NaN
77.0   NaN
78.0   NaN
79.0   NaN
80.0   NaN
Name: std_dev_prices, dtype: float64

In [2]:
# Optimization
######################

maxiters = 2 #120. About 2 minutes per iteration

def error_w_data(θ) -> float:
    return src.gmm_error(θ, policyF, xs,
                         mean_std_observed_prices=mean_std_observed_prices, df=df,
                         prior_shocks=prior_shocks, min_periods=min_periods)

optimi = opt.differential_evolution(error_w_data, [(-2.5, 0.5), (2.0, 4.0),
                                                   (0.5, 2), (-3., 1.)],
                                    maxiter=maxiters)

../src/gmmtools.py:48: RuntimeWarning: overflow encountered in double_scalars
  H = np.e**((θ[0] + θ[1]*x + prior_shock))
../src/gmmtools.py:49: RuntimeWarning: overflow encountered in double_scalars
  Eβ = -np.e**(θ[2] + θ[3]*x + prior_shock) #Bound it?


AssertionError: (78, 76)